In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## FIXED PLANNING HORIZONS
### IMPORT DATA

In [30]:
# forecasts
data = pd.read_csv('1.csv', sep=";")
data

,period,forecast
0,1,250
1,2,170
2,3,960
3,4,80
4,5,10
5,6,541
6,7,230
7,8,112
8,9,180
9,10,298


### METHOD TO SOLVE: WAGNER-WHITIN

#### FORWARD

In [31]:
# Create columns
data_calc = data.copy()

for i in data_calc['period'].unique():
    data_calc['Order {}'.format(i)] = 0

# costs
set_up = 500
holding = 1

# Order 1
order = 1
for index, row in data_calc.iterrows():
    current_month = data_calc.loc[index,'period']
    cost = 0
    # 1 set up
    cost += set_up
    if current_month > 1:
        for t in range(1, current_month+1):
            cost += (t-1) * data_calc.loc[t-1,'forecast'] * holding
    data_calc.loc[index,'Order {}'.format(order)] = cost

# Order 2
for order in range(2, 13):
    for index, row in data_calc.iterrows():
        current_month = data_calc.loc[index,'period']
        if current_month >= order:
            cost = 0

            # Best option best Period 1
            values = list(data_calc.loc[order-2,['Order {}'.format(i) for i in range(1, order+1)]].values)
            best = min([i for i in values if i >0])

            # Add 
            cost += best + set_up
            for t in range(order, current_month+1):
                cost += (t-order) * data_calc.loc[t-1,'forecast'] * holding
            data_calc.loc[index,'Order {}'.format(order)] = cost

data_calc = data_calc.set_index('period').drop(['forecast'], axis = 1).T
data_calc

period,1,2,3,4,5,6,7,8,9,10,11,12
Order 1,500,670,2590,2830,2870,5575,6955,7739,9179,11861,15661,26111
Order 2,0,1000,1960,2120,2150,4314,5464,6136,7396,9780,13200,22700
Order 3,0,0,1170,1250,1270,2893,3813,4373,5453,7539,10579,19129
Order 4,0,0,0,1670,1680,2762,3452,3900,4800,6588,9248,16848
Order 5,0,0,0,0,1750,2291,2751,3087,3807,5297,7577,14227
Order 6,0,0,0,0,0,1770,2000,2224,2764,3956,5856,11556
Order 7,0,0,0,0,0,0,2270,2382,2742,3636,5156,9906
Order 8,0,0,0,0,0,0,0,2500,2680,3276,4416,8216
Order 9,0,0,0,0,0,0,0,0,2724,3022,3782,6632
Order 10,0,0,0,0,0,0,0,0,0,3180,3560,5460


#### BACKWARD

In [32]:
costs, initials, nexts, quantities = [], [], [], []
i = 12
while i>1:
    # Order with the minimum cost
    initial_step = i
    next_step = data_calc[data_calc[i]>0][i].idxmin()
    cost = data_calc[data_calc[i]>0][i].min()
    # Next Step 
    next_id = int(next_step.replace('Order ',''))
    i = next_id - 1
    # Quantity
    quantity = data[data['period'].isin(range(next_id, initial_step+1))]['forecast'].sum()
    
    costs.append(cost)
    initials.append(initial_step)
    nexts.append(next_id)
    quantities.append(quantity)
    
df_results = pd.DataFrame({'backward':range(1, len(initials)+1), 
                           'initial':initials, 
                           'nexts':nexts, 
                           'cost':costs,
                           'quantity':quantities}).set_index('backward')
print("Total Cost: {:,}$".format(df_results.cost.sum()))
df_results

Total Cost: 14,730$


,initial,nexts,cost,quantity
backward,,,,
1,12,12,4022,950
2,11,11,3522,380
3,10,9,3022,478
4,8,6,2224,883
5,5,3,1270,1050
6,2,1,670,420


#### CALCULATION

In [33]:
# Holding unit cost per month
hold_cost = 1
# Set Up Cost
setup_cost = 500

# Final Results
results_final = data.copy()

# Production
month_prod = df_results['nexts'].values

prod_dict = dict(zip(month_prod, df_results.quantity.values))

# Values
results_final['production'] = results_final['period'].apply(lambda t: prod_dict[t] if t in month_prod else 0)

# Inventory On Hand
results_final['IOH'] = (results_final['production'] - results_final['forecast']).cumsum()

# Holding Cost
results_final['Holding Cost'] = (results_final['IOH'] * hold_cost)

# Set Up Cost
results_final['Set-Up Costs'] = results_final['production'].apply(lambda t: setup_cost if t > 0 else 0)

# Total Cost
results_final['Total Cost'] = results_final[['Holding Cost', 'Set-Up Costs']].sum(axis = 1)

results_final

,period,forecast,production,IOH,Holding Cost,Set-Up Costs,Total Cost
0,1,250,420,170,170,500,670
1,2,170,0,0,0,0,0
2,3,960,1050,90,90,500,590
3,4,80,0,10,10,0,10
4,5,10,0,0,0,0,0
5,6,541,883,342,342,500,842
6,7,230,0,112,112,0,112
7,8,112,0,0,0,0,0
8,9,180,478,298,298,500,798
9,10,298,0,0,0,0,0
